# Advent of Code 2020


## Day1: 2SUM
Find product of two numbers in a list such that the sum of the two numbers is 2020.

In [4]:
def day01_2sum(xs: list[int], sumval=2020) -> int:
    """2SUM in O(n)"""
    seen = set()
    for x in xs:
        if sumval - x in seen:
            return x * (sumval - x)
        seen.add(x)

    # not found
    return -1

In [5]:
with open("2020/day01_input.txt") as f:
    xs = [int(x) for x in f.readlines()]
    result = day01_2sum(xs)
    print(result)

786811


## Day1: 3SUM
Find product of three numbers in a list such that the sum of the two numbers is 2020.

In [6]:
def day01_3sum(xs: list[int], sumval=2020) -> int:    
    """3SUM in O(N^2)"""
    seen = set()
    n = len(xs)
    for i in range(n):
        x = xs[i]
        for j in range(i + 1, n):
            y = xs[j]
            z = sumval - x - y
            if z in seen:
                return x * y * z
            seen.add(y)
        seen.add(x)
    
    # not found
    return -1

In [7]:
with open("2020/day01_input.txt") as f:
    xs = [int(x) for x in f.readlines() if x.strip()]
    result = day01_3sum(xs)
    print(result)

199068980


## Day 2: Password validity

For example, `1-3 a` means the letter `a` can appear at least once, and at most three times. Assume each rule is about a single letter usage.

In [7]:
def day02_parse(line: str) -> tuple[int, int, str, str]:
    """
    >>> day02_parse("1-3 a: abcde")
    (1, 3, 'a', 'abcde')
    """
    rule, password = line.split(": ")
    range, letter = rule.split()
    from_, to_ = map(int, range.split("-"))
    return (from_, to_, letter, password)

def day02_isvalid(line: str) -> bool:
    """
    >>> day02_isvalid(1, 3, 'a', 'abcde')
    True
    >>> day02_isvalid(1, 3, 'b', 'cdefg')
    False
    >>> day02_isvalid(2, 9, 'c', 'ccccccccc')
    True
    """
    from_, to_, letter, password = day02_parse(line)
    cnt = password.count(letter)
    return from_ <= cnt and cnt <= to_

In [8]:
with open("2020/day02_input.txt") as f:
    xs = [x for x in f.readlines() if x.strip()]
    result = sum(day02_isvalid(x) for x in xs)
    print(result)

607


In [16]:
def day02_isvalid_updated(line: str) -> bool:
    """
    >>> day02_isvalid_updated(1, 3, 'a', 'abcde')
    True
    >>> day02_isvalid_updated(1, 3, 'b', 'cdefg')
    False
    >>> day02_isvalid_updated(2, 9, 'c', 'ccccccccc')
    False
    """
    from_, to_, letter, password = day02_parse(line)
    criteria1 = password[from_ - 1] == letter
    criteria2 = password[to_ - 1] == letter
    return criteria1 != criteria2

In [17]:
with open("2020/day02_input.txt") as f:
    xs = [x for x in f.readlines() if x.strip()]
    result = sum(day02_isvalid_updated(x) for x in xs)
    print(result)

321


## Day 3: Slope in 2D

In [40]:
def day03_slope(grid: list[str], hstepsize=3, vstepsize=1) -> int:
    """
    Grid is rectangular composed of cells with either '.' or '#'.
    """
    height = len(grid)
    width = len(grid[0])
    maxstep = (height + vstepsize - 1) // vstepsize
    s = "".join(grid[vstepsize * i][(hstepsize * i) % width] for i in range(1, maxstep))
    return s.count("#")

In [41]:
with open("2020/day03_input.txt") as f:
    grid = [line.strip() for line in f.readlines() if line.strip()]
    result = day03_slope(grid)
    print(result)

189


In [42]:
# Day 3 test 
grid = [
    "..##.......",
    "#...#...#..",
    ".#....#..#.",
    "..#.#...#.#",
    ".#...##..#.",
    "..#.##.....",
    ".#.#.#....#",
    ".#........#",
    "#.##...#...",
    "#...##....#",
    ".#..#...#.#",
]

In [46]:
with open("2020/day03_input.txt") as f:
    grid = [line.strip() for line in f.readlines() if line.strip()]
    move11 = day03_slope(grid, 1, 1)
    move31 = day03_slope(grid, 3, 1)
    move51 = day03_slope(grid, 5, 1)
    move71 = day03_slope(grid, 7, 1)
    move12 = day03_slope(grid, 1, 2)

    result = move11 * move31 * move51 * move71 * move12
    print(result)

1718180100


## Day 4: Passport scanning

In [21]:
from typing import Any, Union

def day04_has_fields(data: dict[str, str]) -> bool:
    """
    """
    required_fields = {'byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'}
    return all(f in data for f in required_fields)

def day04_parse(chunk: str) -> dict[str, str]:
    fields = [item.strip() for item in chunk.split() if item.strip()]
    d = dict()
    for field in fields:
        k, v = field.split(':')
        d[k] = v
    return d

In [20]:
with open("2020/day04_input.txt") as f:
    chunks = [chunk.strip() for chunk in f.read().split('\n\n')]
    ds = [day04_parse(chunk) for chunk in chunks]
    records_with_fields = [d for d in ds if day04_has_fields(d)]
    print(len(records_with_fields))

210


In [30]:
def day04_isvalid(x: dict[str, str]) -> bool:
    """
    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.

    """
    if not day04_has_fields(x):
        return False
        
    byr = len(x["byr"]) == 4 and '1920' <= x["byr"] <= '2002'
    iyr = len(x["iyr"]) == 4 and '2010' <= x["iyr"] <= '2020'
    eyr = len(x["eyr"]) == 4 and '2020' <= x["eyr"] <= '2030'
    hgt = (
        (x["hgt"].endswith('cm') and '150' <= x["hgt"][:-2] <= '193') 
        or 
        (x["hgt"].endswith('in') and '59' <= x["hgt"][:-2] <= '76')
    )
    hcl = (
        x["hcl"].startswith('#') and len(x["hcl"][1:]) == 6 and 
        set(x["hcl"][1:]) < set('0123456789abcdef')
    )
    ecl = x["ecl"] in "amb blu brn gry grn hzl oth".split()
    pid = len(x["pid"]) == 9 and x["pid"].isdigit()

    return all([byr, iyr, eyr, hgt, hcl, ecl, pid])
    

In [31]:
result = sum(day04_isvalid(d) for d in records_with_fields)
print(result)

131


In [32]:
test = """
hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022
"""
rec = day04_parse(test)
print(rec)
day04_isvalid(rec)

{'hcl': '#888785', 'hgt': '164cm', 'byr': '2001', 'iyr': '2015', 'cid': '88', 'pid': '545766238', 'ecl': 'hzl', 'eyr': '2022'}


True

## Day 5: Flight seating

In the 7-letter seating code, the first `F`/`B` corresponds to `0`/`1` in the leftmost significant digit in the binary representation, and the second corresponds to the second leftmost significant digit, and so on. Similarly, `L`/`R` works in the exactly the same manner as `F`/`B`. Due to the clever choice of the seat ID, the ID itself is identical to the binary number translated from the sequence of `F`/`B`/`L`/`R` letters.

In [52]:
def day05_decode(code: str) -> int:
    """
    >>> day05_decode("FBFBBFF")
    44

    >>> day05_decode("RLR")
    5

    >>> day05_decode("FBFBBFFRLR")
    357
    """
    assert set(code) <= set("FBLR")
    assert len(code) == 10
    table = str.maketrans(dict(F='0', B='1', L='0', R='1'))
    binseq = code.translate(table)
    return int(binseq, base=2)


In [55]:
day05_decode("FBFBBFFRLR")

357

In [56]:
with open("2020/day05_input.txt") as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    result = max(day05_decode(line) for line in lines)
    print(result)

963


Find the seat ID sandwitched by the existing seats in the list.

In [57]:
with open("2020/day05_input.txt") as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    seat_ids = [day05_decode(line) for line in lines]
    seat_ids.sort()
    for (a, b) in zip(seat_ids, seat_ids[1:]):
        if b - a == 2:
            print(a + 1)

592


## Day 6: union and intersection

A little bit boring.

In [76]:
def day06(lines: str, f=set.union) -> int:
    chunk = lines.strip()
    letters_set = [set(s.strip()) for s in chunk.split()]
    letters = f(*letters_set)
    return len(letters)

In [77]:
with open("2020/day06_input.txt") as f:
    chunks = [chunk.strip() for chunk in f.read().split("\n\n") if chunk.strip()]
    counts = [day06(chunk) for chunk in chunks]
    result = sum(counts)
    print(result)

6590


In [78]:
with open("2020/day06_input.txt") as f:
    chunks = [chunk.strip() for chunk in f.read().split("\n\n") if chunk.strip()]
    counts = [day06(chunk, set.intersection) for chunk in chunks]
    result = sum(counts)
    print(result)

3288


## Day 7: DAG

The bag relation may be represented by directed acyclic graph (DAG). A DAG may be represented by node -> (adjascent nodes). A node is represented by (tone, color).

In [108]:
import collections

Node = tuple[str, str]
Dag = dict[Node, collections.Counter[Node]]

def day07_parse(lines: list[str]) -> Dag:
    """Parse lines and get adjascency lists.
    """
    
    def parse(line: str) -> tuple[Node, collections.Counter[Node]]:
        words = line.split()
        node_orig: Node = (words[0], words[1])
        acc = collections.Counter() 
        if "contain no" in line:
            return (node_orig, acc)

        _, latter = line.split(" bags contain ")
        items = [xs.split() for xs in latter.split(", ")]
        for item in items:
            try:
                count = int(item[0])
                tone = item[1]
                color = item[2]
                node = (tone, color)
                acc.update({node: count})
            except ValueError:
                pass
        return (node_orig, acc)

    d = dict()
    for line in lines:
        node_orig, adj_nodes = parse(line)
        d[node_orig] = adj_nodes

    return d


def day07_part1(g: Dag, target: Node=("shiny", "gold")) -> int:
    """Find number of nodes reachable to the target"""

    def f(curr: Node) -> int:
        if g[curr]:
            if target in g[curr]:
                return g[curr][target]
            else:
                return max(count * f(node) for node, count in g[curr].items())   
        return 0

    return sum(f(node) > 0 for node in g)

In [109]:
test = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""

xs = [x.strip() for x in test.split("\n") if x.strip()]
print(xs)
g = day07_parse(xs)
day07_part1(g)

['light red bags contain 1 bright white bag, 2 muted yellow bags.', 'dark orange bags contain 3 bright white bags, 4 muted yellow bags.', 'bright white bags contain 1 shiny gold bag.', 'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.', 'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.', 'dark olive bags contain 3 faded blue bags, 4 dotted black bags.', 'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.', 'faded blue bags contain no other bags.', 'dotted black bags contain no other bags.']


4

In [110]:
with open("2020/day07_input.txt") as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    g = day07_parse(lines)
    result = day07_part1(g)
    print(result)

254


In [121]:
import functools

def day07_part2(g: Dag, start: Node=("shiny", "gold")) -> int:
    """Find total number of edges in the downstream from the start node"""

    @functools.lru_cache()
    def f(curr: Node) -> int:
        if g[curr]:
            return sum(count * (1 + f(node)) for node, count in g[curr].items())
        return 0

    return f(start)

In [122]:
test = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""

xs = [x.strip() for x in test.split("\n") if x.strip()]
print(xs)
g = day07_parse(xs)
day07_part2(g)

['light red bags contain 1 bright white bag, 2 muted yellow bags.', 'dark orange bags contain 3 bright white bags, 4 muted yellow bags.', 'bright white bags contain 1 shiny gold bag.', 'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.', 'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.', 'dark olive bags contain 3 faded blue bags, 4 dotted black bags.', 'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.', 'faded blue bags contain no other bags.', 'dotted black bags contain no other bags.']


32

In [123]:
with open("2020/day07_input.txt") as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    g = day07_parse(lines)
    result = day07_part2(g)
    print(result)

6006


## Day 8: First assembly language

* `nop`
* `acc`
* `jmp`


In [16]:

def day08_part1(content: str) -> int:
    """
    string-input interface of assembly language execution.
    """
    tuples = [line.strip().split() for line in content.split("\n") if line.strip()]
    program = [(cmd, int(arg)) for cmd, arg in tuples]
    result, _ = day08_run(program)
    return result


def day08_run(program: list[tuple[str, int]]) -> tuple[int, bool]:
    """
    Execute assembly language composed of nop, jmp, and acc.
    """
    n = len(program)
    acc = 0
    i = 0
    seen = set()
    hist = []
    while i not in seen and i < n:
        seen.add(i)
        hist.append(i)
        cmd, arg = program[i]
        if cmd == "nop":
            i += 1
        elif cmd == "acc":
            i += 1
            acc += arg
        elif cmd == "jmp":
            i += arg
        else:
            raise ValueError("Unknown command: {}".format(cmd))

    is_terminated = i == n
    # print(hist)
    return acc, is_terminated


In [17]:
test = """
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
"""

day08_part1(test)

5

In [18]:
with open("2020/day08_input.txt") as f:
    result = day08_part1(f.read())
    print(result)

1610


### Day08: Part2
Replace single `jmp` with `nop` such that the program terminates. Here, program termination is meant by reaching to the last instruction (and finishing it).

The assembly program may be considered as a directed graph; a node is a tuple of head and tail indices, and edge is created by `jmp` operation. Each node has a single outgoing edge. Replacing `jmp` to `nop` corresponds to replacing an edge to another.

But the data contains merely 223 number of jumps hence naive search works.

In [21]:
from typing import Generator

AssemblyProgram = list[tuple[str, int]]

def day08_part2(content: str) -> int:
    """
    
    """
    def gen_jmp2nop(program: AssemblyProgram) -> Generator[AssemblyProgram, None, None]:
        n = len(program)
        for i in range(n):
            if program[i][0] == "jmp":
                new_program = program[:]
                new_program[i] = ("nop", program[i][1])
                yield new_program

    tuples = [line.strip().split() for line in content.split("\n") if line.strip()]
    program = [(cmd, int(arg)) for cmd, arg in tuples]
    for program_mod in gen_jmp2nop(program):
        acc, is_terminated = day08_run(program_mod)
        if is_terminated:
            return acc

    # failed
    return -10000000

In [22]:
with open("2020/day08_input.txt") as f:
    result = day08_part2(f.read())
    print(result)

1703
